In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.stats as ss

from importlib import reload
from neuropacks import RET2
from noise_correlations import analysis, plot, utils

%matplotlib inline

In [ ]:
base_path = '/Volumes/pss'

In [ ]:
data_path = os.path.join(base_path, 'data/ret2/200114_fov1_data.mat')
ret2 = RET2(data_path=data_path)

In [ ]:
X = ret2.get_response_matrix(cells='all', response='max')
stimuli = ret2.angles
unique_stimuli = ret2.unique_angles

# Characterizing response distributions

In [ ]:
peak_responses = utils.get_peak_response(X, stimuli)
modulation_fractions = utils.get_tuning_modulation_fraction(X, stimuli)
selectivity_indices = utils.get_selectivity_index(X, stimuli, circular=360)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3))

axes[0].hist(
    peak_responses,
    histtype='step',
    bins=np.linspace(0, 2.5, 21),
    linewidth=3)
axes[1].hist(
    modulation_fractions,
    histtype='step',
    bins=np.linspace(0, 1, 16),
    linewidth=3)
axes[2].hist(
    selectivity_indices,
    histtype='step',
    bins=np.linspace(0, 0.5, 16),
    linewidth=3)

axes[0].set_xlim([0, 2.5])
axes[1].set_xlim([0, 1])
axes[2].set_xlim([0, 0.5])
axes[0].set_xlabel(r'\textbf{Peak Response}', fontsize=15)
axes[1].set_xlabel(r'\textbf{Modulation Fractions}', fontsize=15)
axes[2].set_xlabel(r'\textbf{Direction Selectivity}', fontsize=15)

for ax in axes:
    ax.set_ylabel(r'\textbf{Frequency}', fontsize=15)
    ax.tick_params(labelsize=15)
    
plt.tight_layout()

# Identifying responsive, tuned, and non-responsive units

In [ ]:
responsive_threshold = 1.0

In [ ]:
tuned_units = np.zeros(X.shape[1]).astype(bool)
tuned_units[ret2.tuned_cells] = True
responsive_units = utils.get_responsive_units(X, stimuli,
                                              aggregator=np.mean,
                                              peak_response=responsive_threshold)
responsive_non_tuned_units = responsive_units & ~tuned_units
non_responsive_units = ~responsive_units

In [ ]:
fig, axes = plot.plot_tuning_curves(
    X[:, modulation_fractions > 0.50], stimuli,
    n_cols=10, include_points=True, use_title=False,
    sort='modulation'
)
for idx, ax in enumerate(axes.ravel()):
    if idx < tuned_units.sum():
        ax.axhline(0, color='gray')
        ax.axhline(responsive_threshold, color='red', linestyle='--')

fig.text(x=0.5, y=1.01, s=r'\textbf{RET2: Tuned Units}',
         va='center', ha='center', fontsize=30)
plt.tight_layout()
# plt.savefig('ret2_tuned.pdf', bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plot.plot_tuning_curves(
    X[:, tuned_units], stimuli,
    n_cols=10, include_points=True, use_title=False,
    sort='modulation'
)
for idx, ax in enumerate(axes.ravel()):
    if idx < tuned_units.sum():
        ax.axhline(0, color='gray')
        ax.axhline(responsive_threshold, color='red', linestyle='--')

fig.text(x=0.5, y=1.01, s=r'\textbf{RET2: Tuned Units}',
         va='center', ha='center', fontsize=30)
plt.tight_layout()
# plt.savefig('ret2_tuned.pdf', bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plot.plot_tuning_curves(
    X[:, responsive_units], stimuli,
    n_cols=10, include_points=True, use_title=False,
    sort='modulation'
)
for idx, ax in enumerate(axes.ravel()):
    if idx < responsive_units.sum():
        ax.axhline(0, color='gray')
        ax.axhline(responsive_threshold, color='red', linestyle='--')

fig.text(x=0.5, y=1.005, s=r'\textbf{Responsive Units}',
         va='center', ha='center', fontsize=30)
plt.tight_layout()
plt.show()

# Noise Correlations

In [ ]:
all_corrs = analysis.all_correlations(X, stimuli)
tuned_corrs = analysis.all_correlations(X, stimuli, u1=tuned_units)
responsive_corrs = analysis.all_correlations(X, stimuli, u1=responsive_units)
responsive_non_tuned_corrs = analysis.all_correlations(X, stimuli, u1=responsive_non_tuned_units)
non_responsive_corrs = analysis.all_correlations(X, stimuli, u1=non_responsive_units)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
bins = np.linspace(-1, 1, 100)
ax.hist(all_corrs, bins=bins,
        density=True,
        histtype='step',
        linewidth=2,
        label='All')
ax.hist(tuned_corrs, bins=bins,
        density=True,
        histtype='step',
        linewidth=2,
        label='Tuned')
ax.hist(responsive_corrs, bins=bins,
        density=True,
        histtype='step',
        linewidth=2,
        label='Responsive')
ax.hist(responsive_non_tuned_corrs, bins=np.linspace(-1, 1, 30),
        density=True,
        histtype='step',
        linewidth=2,
        label='Responsive,\nNon-tuned')
ax.hist(non_responsive_corrs, bins=bins,
        density=True,
        histtype='step',
        linewidth=2,
        label='Non-responsive')

ax.set_xlim([-1, 1])
ax.set_xticks([-1, -0.5, 0, 0.5, 1])
ax.tick_params(labelsize=18)
ax.set_xlabel(r'\textbf{Noise Correlation}', fontsize=18)
ax.set_ylabel(r'\textbf{Density}', fontsize=18)
ax.set_title(r'\textbf{RET2: Within-group correlations}', fontsize=18)
plt.legend(loc=2, prop={'size': 16})
plt.tight_layout()
# plt.savefig('ret2_within_group_correlations.pdf', bbox_inches='tight')
plt.show()

In [ ]:
tuned_by_non_tuned_corrs = analysis.all_correlations(X, stimuli, u1=tuned_units, u2=~tuned_units)
tuned_by_responsive_non_tuned_corrs = analysis.all_correlations(X, stimuli, u1=tuned_units, u2=responsive_non_tuned_units)
responsive_by_non_responsive_corrs = analysis.all_correlations(X, stimuli, u1=responsive_units, u2=~responsive_units)
tuned_by_non_responsive_corrs = analysis.all_correlations(X, stimuli, u1=tuned_units, u2=~responsive_units)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
bins = np.linspace(-1, 1, 100)
ax.hist(all_corrs, bins=bins,
        density=True,
        histtype='step',
        linewidth=2,
        label='All')
ax.hist(tuned_by_non_tuned_corrs, bins=bins,
        density=True,
        histtype='step',
        linewidth=2,
        label='Tuned,\nNon-tuned')
ax.hist(tuned_by_responsive_non_tuned_corrs, bins=bins,
        density=True,
        histtype='step',
        linewidth=2,
        label='Responsive,\nNon-responsive')
ax.hist(responsive_by_non_responsive_corrs, bins=bins,
        density=True,
        histtype='step',
        linewidth=2,
        label='Tuned,\nNon-responsive')
ax.hist(tuned_by_non_responsive_corrs, bins=bins,
        density=True,
        histtype='step',
        linewidth=2,
        label='Tuned,\nResponsive\nnon-tuned')
ax.set_xlim([-1, 1])
ax.set_xticks([-1, -0.5, 0, 0.5, 1])
ax.tick_params(labelsize=18)
ax.set_xlabel(r'\textbf{Noise Correlation}', fontsize=18)
ax.set_ylabel(r'\textbf{Density}', fontsize=18)
ax.set_title(r'\textbf{RET2: Between-group correlations}', fontsize=18)
plt.legend(loc=2, prop={'size': 15})
# plt.savefig('ret2_between_group_correlations.pdf', bbox_inches='tight')
plt.show()